In [1]:
import pandas as pd
import numpy as np  

In [2]:
df_delivery = pd.read_csv('../01_CSV Trabajo/df_delivery_limpio.csv')

In [3]:
df_delivery.head()

,store_primary_category,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders,delivery_duration,partner_density,order_day,order_hour,order_period,busy_ratio,avg_item_price,order_size,grouped_category
0,american,4,3441,4,557,1239,33.0,14.0,21.0,3779.0,1.500000,Friday,22,Night,0.411765,860.25,Medium,American
1,mexican,1,1900,1,1400,1400,1.0,2.0,2.0,4024.0,0.333333,Tuesday,21,Evening,1.000000,1900.00,Small,Mexican
2,indian,4,4771,3,820,1604,8.0,6.0,18.0,1586.0,0.421053,Monday,0,Night,0.666667,1192.75,Medium,Indian
3,italian,1,1525,1,1525,1525,5.0,6.0,8.0,2273.0,0.555556,Thursday,3,Night,1.000000,1525.00,Small,Italian
4,italian,2,3620,2,1425,2195,5.0,5.0,7.0,2988.0,0.625000,Tuesday,2,Night,0.833333,1810.00,Small,Italian


In [4]:
# Verifico las columnas y tipos de datos del dataframe
df_delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166468 entries, 0 to 166467
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   store_primary_category    166468 non-null  object 
 1   total_items               166468 non-null  int64  
 2   subtotal                  166468 non-null  int64  
 3   num_distinct_items        166468 non-null  int64  
 4   min_item_price            166468 non-null  int64  
 5   max_item_price            166468 non-null  int64  
 6   total_onshift_partners    166468 non-null  float64
 7   total_busy_partners       166468 non-null  float64
 8   total_outstanding_orders  166468 non-null  float64
 9   delivery_duration         166468 non-null  float64
 10  partner_density           166468 non-null  float64
 11  order_day                 166468 non-null  object 
 12  order_hour                166468 non-null  int64  
 13  order_period              166468 non-null  o

In [5]:
# Elimino las columnas no relevantes con el fin de obtener un df más limpio
df_delivery = df_delivery.drop(columns=[
    'store_primary_category', 'total_items', 'subtotal', 'num_distinct_items', 
    'min_item_price', 'max_item_price', 'avg_item_price', 'order_size', 'order_period'
])

In [6]:
df_delivery.head()

,total_onshift_partners,total_busy_partners,total_outstanding_orders,delivery_duration,partner_density,order_day,order_hour,busy_ratio,grouped_category
0,33.0,14.0,21.0,3779.0,1.500000,Friday,22,0.411765,American
1,1.0,2.0,2.0,4024.0,0.333333,Tuesday,21,1.000000,Mexican
2,8.0,6.0,18.0,1586.0,0.421053,Monday,0,0.666667,Indian
3,5.0,6.0,8.0,2273.0,0.555556,Thursday,3,1.000000,Italian
4,5.0,5.0,7.0,2988.0,0.625000,Tuesday,2,0.833333,Italian


In [7]:
# Compruebo si hay valores nulos en el df para no tener sorpresas
df_delivery.isnull().sum()


total_onshift_partners      0
total_busy_partners         0
total_outstanding_orders    0
delivery_duration           0
partner_density             0
order_day                   0
order_hour                  0
busy_ratio                  0
grouped_category            0
dtype: int64

In [8]:
# Verifico los tipos de datos del df
df_delivery.dtypes

total_onshift_partners      float64
total_busy_partners         float64
total_outstanding_orders    float64
delivery_duration           float64
partner_density             float64
order_day                    object
order_hour                    int64
busy_ratio                  float64
grouped_category             object
dtype: object

In [9]:
# Convierto la columna 'order_day' a tipo categórico
df_delivery['order_day'] = df_delivery['order_day'].astype('category')

In [10]:
# Calculo el número de repartidores necesarios según la fórmula
df_delivery['repartidores_necesarios'] = df_delivery['total_outstanding_orders'] / df_delivery['partner_density']

In [11]:
# Comparar los repartidores necesarios con los repartidores disponibles
df_delivery['suficientes_repartidores'] = df_delivery['total_onshift_partners'] >= df_delivery['repartidores_necesarios']

In [12]:
# Verifico las primeras filas con los nuevos cálculos
df_delivery[['total_outstanding_orders', 'partner_density', 'repartidores_necesarios', 'total_onshift_partners', 'suficientes_repartidores']].head()

,total_outstanding_orders,partner_density,repartidores_necesarios,total_onshift_partners,suficientes_repartidores
0,21.0,1.500000,14.00,33.0,True
1,2.0,0.333333,6.00,1.0,False
2,18.0,0.421053,42.75,8.0,False
3,8.0,0.555556,14.40,5.0,False
4,7.0,0.625000,11.20,5.0,False


In [13]:
# Agrupo los datos por tipo de restaurante (grouped_category) y calculo el promedio de repartidores necesarios y disponibles
df_grouped = df_delivery.groupby('grouped_category').agg(
    repartidores_necesarios_promedio=('repartidores_necesarios', 'mean'),
    repartidores_disponibles_promedio=('total_onshift_partners', 'mean'),
    pedidos_pendientes_promedio=('total_outstanding_orders', 'mean'),
    densidad_promedio=('partner_density', 'mean')
).reset_index()

In [14]:
# Añado una columna para evaluar la suficiencia promedio de los repartidores en cada tipo de restaurante
df_grouped['promedio_suficientes'] = df_grouped['repartidores_disponibles_promedio'] >= df_grouped['repartidores_necesarios_promedio']

In [15]:
df_grouped

,grouped_category,repartidores_necesarios_promedio,repartidores_disponibles_promedio,pedidos_pendientes_promedio,densidad_promedio,promedio_suficientes
0,American,inf,38.941762,49.082120,inf,False
1,Asian,inf,49.220526,64.579669,NaN,False
2,Beverages,inf,50.985591,64.797310,0.918768,False
3,Desserts,inf,46.231688,60.017143,inf,False
4,European,87.777177,45.087719,59.882591,0.897101,False
5,Fast Food,inf,37.962374,47.428848,inf,False
6,Healthy,inf,44.813365,60.013599,0.889563,False
7,Indian,102.389567,51.513551,69.284108,0.836815,False
8,Italian,inf,50.069045,65.630027,0.853670,False
9,Latin,69.132886,36.470231,47.712029,0.882449,False


In [16]:
# Cuento la cantidad de registros por cada tipo de restaurante
category_counts = df_delivery['grouped_category'].value_counts().reset_index()
category_counts.columns = ['grouped_category', 'count']

In [17]:
# Muestro las categorías con pocos registros
category_counts.sort_values(by='count', ascending=True).head(10)

,grouped_category,count
12,European,741
11,Latin,823
10,Beverages,2082
9,Healthy,4265
8,Indian,6494
7,Desserts,7700
6,Mediterranean,9195
5,Mexican,15623
4,Other,17134
3,American,20811


In [18]:
# Filtro las categorías con pocos registros
pocos_registros_categorias = category_counts[category_counts['count'] <= 1000]['grouped_category']


In [19]:
# Filtrar el dataframe por esas categorías
df_low_count = df_delivery[df_delivery['grouped_category'].isin(pocos_registros_categorias)]

In [20]:
# Ver los valores extremos de las columnas clave en estas categorías
df_low_count[['grouped_category', 'repartidores_necesarios', 'total_onshift_partners', 'partner_density', 'total_outstanding_orders']].describe()

,repartidores_necesarios,total_onshift_partners,partner_density,total_outstanding_orders
count,1556.000000,1564.000000,1564.000000,1564.000000
mean,77.951779,40.553069,0.889391,53.478261
std,93.953828,34.414010,0.404707,54.334232
min,0.000000,0.000000,0.000000,0.000000
25%,16.230525,14.000000,0.649175,14.000000
50%,39.023810,29.000000,0.800000,32.000000
75%,105.551821,57.250000,1.015494,75.000000
max,530.217391,156.000000,5.000000,272.000000


In [21]:
# Defino los rangos aceptables para cada columna
repartidores_necesarios_min = 1
repartidores_necesarios_max = 200

total_onshift_partners_min = 1
total_onshift_partners_max = 100

partner_density_min = 0.1
partner_density_max = 3.0

total_outstanding_orders_min = 1
total_outstanding_orders_max = 100

In [22]:
# Filtro los datos excluyendo los valores fuera de los rangos definidos
df_filtered = df_low_count[
    (df_low_count['repartidores_necesarios'] >= repartidores_necesarios_min) &
    (df_low_count['repartidores_necesarios'] <= repartidores_necesarios_max) &
    (df_low_count['total_onshift_partners'] >= total_onshift_partners_min) &
    (df_low_count['total_onshift_partners'] <= total_onshift_partners_max) &
    (df_low_count['partner_density'] >= partner_density_min) &
    (df_low_count['partner_density'] <= partner_density_max) &
    (df_low_count['total_outstanding_orders'] >= total_outstanding_orders_min) &
    (df_low_count['total_outstanding_orders'] <= total_outstanding_orders_max)
]

In [23]:
# Verifico cuántos registros quedan después de la filtración
df_filtered.shape

(1255, 11)

In [24]:
df_filtered

,total_onshift_partners,total_busy_partners,total_outstanding_orders,delivery_duration,partner_density,order_day,order_hour,busy_ratio,grouped_category,repartidores_necesarios,suficientes_repartidores
730,7.0,7.0,7.0,3362.0,0.875000,Sunday,23,0.875000,Latin,8.000000,False
731,12.0,12.0,19.0,3236.0,0.600000,Monday,2,0.923077,Latin,31.666667,False
946,11.0,11.0,9.0,2849.0,1.100000,Tuesday,2,0.916667,Latin,8.181818,True
947,5.0,5.0,6.0,5382.0,0.714286,Tuesday,1,0.833333,Latin,8.400000,False
948,5.0,5.0,10.0,5464.0,0.454545,Saturday,22,0.833333,Latin,22.000000,False
...,...,...,...,...,...,...,...,...,...,...,...
163270,3.0,2.0,2.0,2173.0,1.000000,Tuesday,3,0.500000,Latin,2.000000,True
163271,7.0,7.0,8.0,5163.0,0.777778,Saturday,2,0.875000,Latin,10.285714,False
163272,7.0,5.0,5.0,1955.0,1.166667,Friday,3,0.625000,Latin,4.285714,True
163273,10.0,8.0,7.0,2080.0,1.250000,Saturday,2,0.727273,Latin,5.600000,True


In [25]:
df_delivery

,total_onshift_partners,total_busy_partners,total_outstanding_orders,delivery_duration,partner_density,order_day,order_hour,busy_ratio,grouped_category,repartidores_necesarios,suficientes_repartidores
0,33.0,14.0,21.0,3779.0,1.500000,Friday,22,0.411765,American,14.000000,True
1,1.0,2.0,2.0,4024.0,0.333333,Tuesday,21,1.000000,Mexican,6.000000,False
2,8.0,6.0,18.0,1586.0,0.421053,Monday,0,0.666667,Indian,42.750000,False
3,5.0,6.0,8.0,2273.0,0.555556,Thursday,3,1.000000,Italian,14.400000,False
4,5.0,5.0,7.0,2988.0,0.625000,Tuesday,2,0.833333,Italian,11.200000,False
...,...,...,...,...,...,...,...,...,...,...,...
166463,17.0,17.0,23.0,3907.0,0.708333,Tuesday,0,0.944444,Fast Food,32.470588,False
166464,12.0,11.0,14.0,3383.0,0.800000,Friday,0,0.846154,Fast Food,17.500000,False
166465,39.0,41.0,40.0,3008.0,0.951220,Saturday,4,1.025000,Fast Food,42.051282,False
166466,7.0,7.0,12.0,3907.0,0.538462,Sunday,18,0.875000,Fast Food,22.285714,False


In [26]:
# Calculo los repartidores necesarios y la suficiencia para todas las categorías
df_delivery['repartidores_necesarios'] = df_delivery['total_outstanding_orders'] / df_delivery['partner_density']


In [27]:
# Comparo los repartidores necesarios con los repartidores disponibles
df_delivery['suficientes_repartidores'] = df_delivery['total_onshift_partners'] >= df_delivery['repartidores_necesarios']


In [28]:
# Agrupo por tipo de restaurante y calcular los promedios
df_agrupado = df_delivery.groupby('grouped_category').agg(
    repartidores_necesarios_promedio=('repartidores_necesarios', 'mean'),
    repartidores_disponibles_promedio=('total_onshift_partners', 'mean'),
    pedidos_pendientes_promedio=('total_outstanding_orders', 'mean'),
    densidad_promedio=('partner_density', 'mean'),
    suficiencia_promedio=('suficientes_repartidores', 'mean')
).reset_index()

In [29]:
# Filtro los registros donde la densidad de repartidores o los pedidos pendientes sean cero
df_delivery_clean = df_delivery[
    (df_delivery['partner_density'] > 0) & 
    (df_delivery['total_outstanding_orders'] > 0)
]

In [31]:
# Vuelvo a calcular los repartidores necesarios y la suficiencia
df_delivery_clean['repartidores_necesarios'] = df_delivery_clean['total_outstanding_orders'] / df_delivery_clean['partner_density']
df_delivery_clean['suficientes_repartidores'] = df_delivery_clean['total_onshift_partners'] >= df_delivery_clean['repartidores_necesarios']


/var/folders/41/9xs275yn7jxbs_h59j3pq3sw0000gn/T/ipykernel_67178/979218612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_delivery_clean['repartidores_necesarios'] = df_delivery_clean['total_outstanding_orders'] / df_delivery_clean['partner_density']
/var/folders/41/9xs275yn7jxbs_h59j3pq3sw0000gn/T/ipykernel_67178/979218612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_delivery_clean['suficientes_repartidores'] = df_delivery_clean['total_onshift_partners'] >= df_delivery_clean['repartidores

In [32]:
# Agrupo por tipo de restaurante y calcular los promedios
df_grouped_clean = df_delivery_clean.groupby('grouped_category').agg(
    repartidores_necesarios_promedio=('repartidores_necesarios', 'mean'),
    repartidores_disponibles_promedio=('total_onshift_partners', 'mean'),
    pedidos_pendientes_promedio=('total_outstanding_orders', 'mean'),
    densidad_promedio=('partner_density', 'mean'),
    suficiencia_promedio=('suficientes_repartidores', 'mean')
).reset_index()

In [34]:
df_delivery_clean

,total_onshift_partners,total_busy_partners,total_outstanding_orders,delivery_duration,partner_density,order_day,order_hour,busy_ratio,grouped_category,repartidores_necesarios,suficientes_repartidores
0,33.0,14.0,21.0,3779.0,1.500000,Friday,22,0.411765,American,14.000000,True
1,1.0,2.0,2.0,4024.0,0.333333,Tuesday,21,1.000000,Mexican,6.000000,False
2,8.0,6.0,18.0,1586.0,0.421053,Monday,0,0.666667,Indian,42.750000,False
3,5.0,6.0,8.0,2273.0,0.555556,Thursday,3,1.000000,Italian,14.400000,False
4,5.0,5.0,7.0,2988.0,0.625000,Tuesday,2,0.833333,Italian,11.200000,False
...,...,...,...,...,...,...,...,...,...,...,...
166463,17.0,17.0,23.0,3907.0,0.708333,Tuesday,0,0.944444,Fast Food,32.470588,False
166464,12.0,11.0,14.0,3383.0,0.800000,Friday,0,0.846154,Fast Food,17.500000,False
166465,39.0,41.0,40.0,3008.0,0.951220,Saturday,4,1.025000,Fast Food,42.051282,False
166466,7.0,7.0,12.0,3907.0,0.538462,Sunday,18,0.875000,Fast Food,22.285714,False


In [35]:
df_delivery_clean.to_csv('df_delivery_repartidores.csv', index=False)

***Empiezo con el Machine Learning***

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

In [37]:
# Selecciono las características y la variable objetivo
X = df_delivery[['order_hour', 'grouped_category', 'total_outstanding_orders', 'partner_density']]
y = df_delivery['total_onshift_partners']  # Nuestro objetivo es predecir la cantidad de repartidores


In [38]:
# Divido los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**¿Qué vamos a usar?**
- Pipeline: Agrupa los pasos del flujo de trabajo en un solo objeto
- SimpleImputer: Es un transformador para manejar nulos y sustituye los nulos por la media
- Onehotencoder: que convierte las variables categóricas como tipo de restaurante en 0 y 1
- RandomForestRegresor: Árboles de decisión.

In [39]:
# Creamos un pipeline con un preprocesador y un modelo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['total_outstanding_orders', 'partner_density']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['order_hour', 'grouped_category'])
    ])

In [40]:
# Crear el pipeline con RandomForestRegressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [41]:
# Verifico si hay valores infinitos o extremadamente grandes en los datos
df_delivery_clean.replace([np.inf, -np.inf], np.nan, inplace=True)


/var/folders/41/9xs275yn7jxbs_h59j3pq3sw0000gn/T/ipykernel_55111/696544028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_delivery_clean.replace([np.inf, -np.inf], np.nan, inplace=True)


In [42]:
# Compruebo si existen valores nulos después de reemplazar los infinitos
df_delivery_clean.isnull().sum()

total_onshift_partners      0
total_busy_partners         0
total_outstanding_orders    0
delivery_duration           0
partner_density             0
order_day                   0
order_hour                  0
busy_ratio                  0
grouped_category            0
repartidores_necesarios     0
suficientes_repartidores    0
dtype: int64

In [43]:
# Divido los datos en entrenamiento y prueba
X = df_delivery_clean[['order_hour', 'grouped_category', 'total_outstanding_orders', 'partner_density']]
y = df_delivery_clean['total_onshift_partners']  # Nuestro objetivo es predecir la cantidad de repartidores

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [45]:
# Entrenar el modelo
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['total_outstanding_orders',
                                                   'partner_density']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['order_hour',
                                                   'grouped_category'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [46]:
# Realizo predicciones sobre el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

In [47]:
# Evaluo el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [48]:
mse, rmse, r2


(0.0336778817157979, np.float64(0.1835153446330794), 0.9999710898110472)

**Valido los resultados con Crossvalidation**

In [49]:
from sklearn.model_selection import cross_val_score

In [50]:
# Aplicar la validación cruzada con 5 folds
#cv_scores = cross_val_score(model_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')

In [51]:
# Calculamos el error cuadrático medio (RMSE) a partir de los resultados de la validación cruzada
cv_rmse = np.sqrt(-cv_scores)

NameError: name 'cv_scores' is not defined

In [96]:
# Calculo la media y la desviación estándar del RMSE
cv_rmse_mean = np.mean(cv_rmse)
cv_rmse_std = np.std(cv_rmse)

In [97]:
cv_rmse_mean, cv_rmse_std

(np.float64(0.17439458432322674), np.float64(0.01289813922479458))

*El modelo tiene un rendimiento consistente*

In [52]:
import joblib

In [53]:
# Guardo el modelo entrenado
joblib.dump(model_pipeline, 'calculo_repartidores.pkl')

['calculo_repartidores.pkl']

In [54]:
import pickle

# Suponiendo que model_pipeline es el modelo entrenado
with open('calculo_repartidores.pkl', 'wb') as f:
    pickle.dump(model_pipeline, f)

In [55]:
# Guardar el modelo entrenado
joblib.dump(model_pipeline, 'calculo_repartidores.pkl')

['calculo_repartidores.pkl']

In [58]:
# Cargar el archivo .pkl
with open('calculo_repartidores.pkl', 'rb') as file:
    model = pickle.load(file)

# Verificar el tipo de objeto cargado
print(type(model))

<class 'numpy.ndarray'>


In [61]:
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

# Crear el pipeline del modelo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['total_outstanding_orders', 'partner_density']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['order_hour', 'grouped_category'])
    ])

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Entrenar el modelo (suponiendo que ya tienes X_train y y_train)
model_pipeline.fit(X_train, y_train)

# Guardar el modelo
joblib.dump(model_pipeline, 'calculo_repartidores.pkl')


['calculo_repartidores.pkl']